In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
import pathlib
from skimage.measure import regionprops, label
from pathlib import Path
import torch
import cv2
import edt
import math
import matplotlib as mpl
from math import floor
%matplotlib qt

In [3]:
filepath = Path('/home/pk/Documents/rtseg/data/backbone/000_img_masks.tif')

In [4]:
image = imread(filepath)

In [5]:
plt.figure()
plt.imshow(image)
plt.show()

#### __Compute the backbone of each region on the image__

#### Fixed parameters

In [6]:
props = regionprops(label(image))

In [7]:
props[0].bbox, props[0].label

((10, 16, 41, 36), 1)

In [8]:
plt.figure()
plt.imshow(props[0].image)
plt.show()

In [9]:
dilation_threshold = 5
dilation_radius = floor(dilation_threshold/2)

In [10]:
dilation_radius

2

In [11]:
img = props[0].image

In [12]:
img.dtype

dtype('bool')

In [13]:
padded_img = np.pad(img, (dilation_radius+1, dilation_radius+1)).astype('uint8')

In [14]:
img.shape, padded_img.shape

((31, 20), (37, 26))

In [15]:
se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dilation_threshold, dilation_threshold))

In [16]:
se

array([[0, 0, 1, 0, 0],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0]], dtype=uint8)

In [17]:
dilated_img = cv2.dilate(padded_img, se)

In [18]:
dilated_img.shape

(37, 26)

In [19]:
xor = np.logical_xor(padded_img, dilated_img)

In [20]:
plt.imshow(xor)

In [21]:
plt.imshow(dilated_img)

In [22]:
region_edt = edt.edt(dilated_img)

In [23]:
plt.imshow(region_edt)

In [24]:
blob_boundary = (region_edt == 1)

In [25]:
blob_boundary.shape

(37, 26)

In [26]:
blob_boundary = blob_boundary[dilation_radius+1: -dilation_radius-1, dilation_radius+1: -dilation_radius-1]

In [27]:
y, x = np.nonzero(blob_boundary)

In [28]:
y, x

(array([ 0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  8,  9,  9, 10, 10, 11, 11,
        12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 30]),
 array([10, 11, 12, 12, 13, 14, 14, 14, 15,  0, 15,  1, 16,  1, 16,  2, 17,
         2, 17,  3, 17,  3, 18,  4, 18,  4, 19,  4, 19,  5, 19,  5,  5,  6,
         6,  6,  7,  7,  7,  8,  8,  9, 10, 19]))

In [29]:
label_dist = region_edt * (region_edt >= dilation_threshold)

In [30]:
final_labelled_image = label_dist[dilation_radius+1: -dilation_radius-1, dilation_radius+1: -dilation_radius-1]

In [31]:
final_labelled_image.shape

(31, 20)

In [32]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(props[0].image)
ax[1].imshow(final_labelled_image)
plt.show()

In [33]:
y_coords, x_coords = np.nonzero(final_labelled_image)

In [34]:
pixel_values = final_labelled_image[y_coords, x_coords]

In [35]:
y_coords.shape, x_coords.shape, pixel_values.shape

((152,), (152,), (152,))

#### I am changing this cubing to a parameter, as I notice some of them are not great

In [36]:
weights = np.sqrt(pixel_values ** 3)

In [37]:
weights.shape

(152,)

In [38]:
A = np.vstack((x_coords**2, x_coords, np.ones(x_coords.shape)))

In [39]:
A.shape

(3, 152)

In [40]:
w_rep = np.tile(weights, [3, 1])

In [41]:
wA = (w_rep * A).T
wB = weights * y_coords

In [42]:
wA.shape, wB.shape

((152, 3), (152,))

In [43]:
from numpy.linalg import lstsq

In [44]:
fit_coeff,_, _, _ = lstsq(wA, wB, rcond=-1)

In [45]:
fit_coeff

array([ 0.02613278,  1.41718813, -3.12621596])

In [46]:
plt.figure()
plt.imshow(final_labelled_image)
x = np.linspace(0.5, final_labelled_image.shape[1]-0.5, num=100)
y = fit_coeff[0] * x**2 + fit_coeff[1] * x + fit_coeff[2]
plt.plot(x, y, 'r-')
plt.show()

#### Compute backbone pixels

In [47]:
img.shape

(31, 20)

In [48]:
img_size = img.shape

In [49]:
x_data = np.arange(-0.5, img_size[1]+0.5)
y_data = fit_coeff[0] * x_data**2 + fit_coeff[1] * x_data + fit_coeff[2]

In [50]:
plt.figure()
plt.imshow(img)
plt.plot(x_data, y_data, 'r--')
plt.show()

##### There are cases. one for a straight line and the other for a quadratic curve

In [51]:
if abs(fit_coeff[0]) < 1e-4 and round(y_data[0]) == round(y_data[-1]):
    fit_coord = np.vstack((np.arange(0, img_size[1]), round(y_data[0]) * np.ones((img_size[1],)))).T
else:
    y_data_round = np.round(y_data)
    stat_pt = -fit_coeff[1] / (2 * fit_coeff[0]) #  ax^2 + bx + c = 0 --> -b/2a is the point where derivative==0
    if stat_pt > -0.5 and stat_pt < img_size[1] - 1:
        round_stat_pt_val = round(fit_coeff[0] * (stat_pt**2) + fit_coeff[1] * stat_pt + fit_coeff[2])
        round_stat_pt = round(stat_pt)
        upp_bound = np.zeros((img_size[1]),) # upper bound defined for each pixel on the x-axis
        low_bound = np.zeros((img_size[1]),) # lower bound defined for each pixel on the x-axis
    
        if fit_coeff[0] > 0:
            upp_bound[0:round_stat_pt] = y_data_round[0:round_stat_pt]
            upp_bound[round_stat_pt] = max(y_data_round[round_stat_pt:round_stat_pt+2])
            upp_bound[round_stat_pt+1:] = y_data_round[round_stat_pt+1:]
    
            low_bound[0:round_stat_pt] = y_data_round[1:round_stat_pt+1]
            round_down_point = np.argwhere(y_data_round[1:] - y_data[1:] == 0.5)
            low_bound[round_down_point] = lowBound[round_down_point]-1 
            low_bound[round_stat_pt] = round_stat_pt_val 
            low_bound[round_stat_pt+1:] =  y_data_round[round_stat_pt+1:]
                                           
        else:
            upp_bound[0:round_stat_pt] = y_data_round[1:round_stat_pt+1]
            upp_bound[round_stat_pt] = round_stat_pt_val
            upp_bound[round_stat_pt+1:] = y_data_round[round_stat_pt:-1];
            low_bound[0:round_stat_pt] =  y_data_round[0:round_stat_pt]
            low_bound[round_stat_pt] = max(y_data_round[round_stat_pt:round_stat_pt+2])
            low_bound[round_stat_pt+1:] = y_data_round[round_stat_pt+1:]
    else:
        if (fit_coeff[0] > 0 and stat_pt > img_size[1]-1) or (fit_coeff[0] < 0 and stat_pt < -0.5):
            upp_bound = y_data_round[:-1]
            low_bound = y_data_round[1:]
            round_down_point = np.argwhere(low_bound - y_data[1:] == 0.5)
            low_bound[round_down_point] = low_bound[round_down_point]-1;
        else:
            upp_bound = y_data_round[1:]
            low_bound = y_data_round[:-1]
        
    
    # clip the upper bound and lower bounds
    # and populate the pixels
    low_bound = np.maximum(low_bound, 0).astype('int')
    upp_bound = np.minimum(upp_bound, img_size[0]-1).astype('int')
    n_pixels_per_col = upp_bound - low_bound + 1
    n_pixels_per_col[n_pixels_per_col < 1] = 0
    fit_coord = np.zeros((int(np.sum(n_pixels_per_col)), 2))
    idx = 0;
    for i in range(len(n_pixels_per_col)):
        fit_coord[idx:idx+n_pixels_per_col[i], 0] = i;
        fit_coord[idx:idx+n_pixels_per_col[i], 1] = np.arange(low_bound[i], upp_bound[i]+1)
        idx += n_pixels_per_col[i]
    
    fit_coord = fit_coord.astype('int')

### Poles

In [52]:
fit_im = np.zeros(img_size, dtype='bool')
fit_im[fit_coord[:, 1], fit_coord[:, 0]] = True
backbone_im = img * fit_im

y_backbone, x_backbone = np.nonzero(backbone_im)

In [53]:
poles = np.zeros((2, 2))

In [54]:
poles[:, 0] = [x_backbone[0] - 0.5, x_backbone[-1] + 0.5]
poles[:, 1] = fit_coeff[0] * poles[:, 0]**2 + fit_coeff[1] * poles[:, 0] + fit_coeff[2]

In [55]:
poles

array([[ 1.5       , -0.94163503],
       [17.5       , 29.67773931]])

In [56]:
plt.figure()
plt.imshow(img)
plt.plot(poles[:, 0], poles[:, 1], '*')
plt.show()

### Arc length

In [57]:
x_0, x_end = x_backbone[0] - 0.5, x_backbone[-1] + 0.5

if abs(fit_coeff[0]) > 1e-10:
    transf_points = (fit_coeff[0] / abs(fit_coeff[0])) * np.array([2*fit_coeff[0]*x_0 + fit_coeff[1] , 2*fit_coeff[0]*x_end + fit_coeff[1]])
    sqrt_exp = np.sqrt(1 + transf_points**2)
    arc_length = np.diff(transf_points * sqrt_exp + np.log(transf_points + sqrt_exp)) / (4 * abs(fit_coeff[0]))
else:
    
    fit_points = [fit_coeff[1] * x_0 + fit_coeff[2], fit_coeff[1] * x_end + fit_coeff[2]]
    arc_length = math.sqrt((x_0 - x_end) ** 2 + (fit_points[0] - fit_points[1]) ** 2)
    

In [63]:
print(arc_length)

[34.59481994]



### Just do a dilation and see if a dot lands on the cell and assign the dot coordinates. Just that simple.
#### Be careful and loop over all the dots as each cell can have more than one dot.

### Compute internal coordinates

In [64]:
poles

array([[ 1.5       , -0.94163503],
       [17.5       , 29.67773931]])

In [65]:
dot_coordinate = np.array([[8.0, 15.0]])

In [66]:
plt.figure()
plt.imshow(img)
plt.plot(x_data, y_data, 'r--')
plt.plot(poles[:, 0], poles[:, 1], '*')
plt.plot(dot_coordinate[:, 0], dot_coordinate[:, 1], 'go')
plt.show()

In [71]:
dot_coordinate

array([[ 8., 15.]])

In [72]:
dot_coordinate.shape

(1, 2)

In [73]:
proj = np.zeros_like(dot_coordinate)

In [74]:
proj

array([[0., 0.]])

#### Project normal and solve a cubic equation

In [68]:
dot_no = 0
p0, p1, p2 = fit_coeff
x0, y0 = dot_coordinate[dot_no]
projected_point = np.zeros(dot_coordinate[dot_no].shape)
if abs(p0) > 1e-10:
    x_star = np.roots([2*p0**2, 3*p0*p1, 2*p0*(p2-y0) + (p1**2+1), p1*(p2-y0)-x0]) # 3rd degree polynomial
    x_star[abs(np.imag(x_star)) < 1e-10] = np.real(x_star[abs(np.imag(x_star)) < 1e-10])
    #print(x_star)
    x_star = np.real(x_star[np.isreal(x_star)])
    x_mid = -p1/(2*p0)
    if x0 > x_mid:
        projected_point[0] = np.max(x_star)
    else:
        projected_point[0] = min(x_star)
else:
    projected_point[0] = x0 - ((p1 * (p2 - p1)) / ((p1**2 + 1)))

projected_point[1] = p0 * projected_point[0] ** 2 + p1 * projected_point[0] + p2
# you should return projected point and distance to projected point.

distance_to_proj = math.sqrt((projected_point[0] - x0)**2 + (projected_point[1] - y0)**2)

In [69]:
plt.figure()
plt.imshow(img)
plt.plot(x_data, y_data, 'r--')
plt.plot(poles[:, 0], poles[:, 1], '*')
plt.plot(dot_coordinate[:, 0], dot_coordinate[:, 1], 'go')
plt.plot(projected_point[0], projected_point[1], 'k*')
plt.plot([dot_coordinate[0, 0], projected_point[0]], [dot_coordinate[0, 1], projected_point[1]], 'b--')
plt.show()

In [70]:
distance_to_proj

2.391979433342466

### Assigning internal coordinates

### Final assemble

In [85]:
from rtseg.cells.regionprops_custom import compute_fit_coeff_poles_arc_length, regionprops_custom
from skimage.measure import regionprops, label
from skimage.io import imread
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('QtAgg')
%matplotlib qt

In [86]:
filepath = Path('/home/pk/Documents/rtseg/data/backbone/img40_20_13.tif')

In [87]:
image = imread(filepath)

In [88]:
plt.figure()
plt.imshow(image)
plt.show()

In [89]:
label_img = label(image)

In [90]:
props = regionprops(label_img, extra_properties=(compute_fit_coeff_poles_arc_length,))

In [93]:
props_custom = regionprops_custom(label_img)

In [94]:
len(props_custom)

1430

In [95]:
props_custom[123].arc_length

array([24.00633763])

In [99]:
cellprop = props_custom[0]

In [100]:
cellprop.fit_coeff, cellprop.arc_length,

array([ 0.01181161, -0.21374283,  3.90755213])

In [87]:
%%timeit 
for cellprop in props:
    cellprop.fit_coeff, cellprop.arc_length, cellprop.poles, cellprop.boundary_pixels = cellprop.compute_fit_coeff_poles_arc_length
    break

448 µs ± 7.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [88]:
for cellprop in props:
    print(cellprop.label, '--->', cellprop.arc_length)

1 ---> [18.136178]


AttributeError: '<class 'skimage.measure._regionprops.RegionProperties'>' object has no attribute 'arc_length'

In [225]:
for i in range(len(props)):
    fit_coeff, arc_length, poles, backbone_coordinates = compute_fit_coeff_poles_arc_length(props[i].image, distance_threshold=3, plot=True)
    #print(f"{props[i].label} ---> {arc_length}")
#print(f"Fit_coeff: {fit_coeff}")
#print(f"Arc length: {arc_length}")
#print(f"Poles: {poles}")
#print(f"Backbone coordinates: {backbone_coordinates}")
#print(props[i].label)


448 ---> [34.35111691]
462 ---> [40.06215628]
489 ---> [30.25122497]
503 ---> [31.16483879]
510 ---> [32.74700458]
513 ---> [31.60069338]
528 ---> [33.3132707]
548 ---> [26.446337]
554 ---> [33.03286623]
557 ---> [30.45956885]
570 ---> [17.89333099]
579 ---> [12.40110883]
590 ---> [32.33814598]
603 ---> [25.11792718]
619 ---> [27.88268215]
631 ---> [36.59609807]


In [220]:
print(props[i].label)

579
